In [ ]:
!wget https://fadedave.com/bees.tar.gz
!tar xf bees.tar.gz
!rm bees.tar.gz

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers, callbacks
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

batch_size = 32
img_height = 180
img_width = 180
img_channels = 3
data_dir = "Bees"

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
	data_dir,
	validation_split=0.2,
	subset="training",
	seed=123,
	image_size=(img_height, img_width),
	batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
	data_dir,
	validation_split=0.2,
	subset="validation",
	seed=123,
	image_size=(img_height, img_width),
	batch_size=batch_size)

class_names = train_ds.class_names
print(class_names)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
data_augmentation = keras.Sequential(
	[
		layers.RandomFlip(input_shape=(img_height, img_width, img_channels)),
		layers.RandomRotation(0.5),
		layers.RandomZoom(0.1),
		layers.RandomTranslation(0.1, 0.1),
	]
)

num_classes = len(class_names)

model = Sequential([
	data_augmentation,
	layers.Rescaling(1./255),
	layers.Conv2D(16, 3, padding='same', activation='relu'),
	layers.MaxPooling2D(),
	layers.Conv2D(32, 3, padding='same', activation='relu'),
	layers.MaxPooling2D(),
	layers.Conv2D(64, 3, padding='same', activation='relu'),
	layers.MaxPooling2D(),
	layers.Dropout(0.2),
	layers.Flatten(),
	layers.Dense(128, activation='relu'),
	layers.Dense(num_classes, name="outputs")
])

model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

model.summary()

epochs = 1
annealer = callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.995 ** (x+epochs))
checkpointer = callbacks.ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, save_weights_only=True)

train_model = model.fit(
	train_ds,
	validation_data=val_ds,
	epochs=epochs,
	callbacks = [ annealer, checkpointer ]
)

In [ ]:
# Validate results
acc = train_model.history['accuracy']
val_acc = train_model.history['val_accuracy']

loss = train_model.history['loss']
val_loss = train_model.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [39]:
model.load_weights("bees_best.h5")  # ez csak akkor kell ha offsite traineled a modelt

In [ ]:
import json, base64, requests
bees = requests.get("http://10.10.7.11:53369/images").text

for i in range(1):
	out = []
	for e,i in enumerate(json.loads(bees)):
		k,v = list(i.items())[0]
		with open(f"bee.png", "wb") as filp:
			filp.write(base64.b64decode(v))
		img = tf.keras.utils.load_img(
			"bee.png", target_size=(img_height, img_width)
		)
		img_array = tf.keras.utils.img_to_array(img)
		img_array = tf.expand_dims(img_array, 0)

		predictions = model.predict(img_array)
		score = tf.nn.softmax(predictions[0])

		print(
			"This image most likely belongs to {} with a {:.2f} percent confidence."
			.format(class_names[np.argmax(score)].replace("_"," "), 100 * np.max(score))
		)

		# out.append({k: class_names[np.argmax(score)].replace("_"," ").replace("Carniolan","Carnolian")})
		out.append(f'"{k}":"{class_names[np.argmax(score)].replace("_"," ")}"')

	# out = json.dumps(out, separators=(',',':'))+"\n"
	out = "{"+",".join(out)+"}\n"

	print(requests.post("http://10.10.7.11:53369/submit", data=out).text)

!rm bee.png